In [1]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

okt = Okt()

In [3]:
from google.colab import files
myfile = files.upload()

Saving 네이버시리즈영화정보_청불포함.csv to 네이버시리즈영화정보_청불포함.csv


In [4]:
data = pd.read_csv('/content/네이버시리즈영화정보_청불포함.csv', encoding='utf-8')
data.head()

,Unnamed: 0,제목,본문
0,0,까불지마,벽돌과 개떡 그들은 한 때 잘나가던 동방파의 서열 23위를 다투는 사이 삼복은 그들...
1,1,엘리트,어두운 학원 교실 조기교육을 받고 있는 아이들이 시험 문제를 풀고 있다 시험이 끝났...
2,2,밀정 1930,동양의 파리로 알려진 화려한 1930년대 상해 그러나 화려한 노래와 춤의 이면에는 ...
3,3,엠티맨,첫날 밤 그의 소리가 들리고 둘째 날 밤 그의 모습이 보이고 셋째 날 밤 그가 너를...
4,5,바이크 원정대 인 이탈리아,이탈리아 남부 살레르노를 중심으로바이커들의 로망 이탈리아를 달린다해외여행에 대한 설...


In [5]:
# drop_duplicates()
data = data.drop_duplicates(['제목'])
data.head()

,Unnamed: 0,제목,본문
0,0,까불지마,벽돌과 개떡 그들은 한 때 잘나가던 동방파의 서열 23위를 다투는 사이 삼복은 그들...
1,1,엘리트,어두운 학원 교실 조기교육을 받고 있는 아이들이 시험 문제를 풀고 있다 시험이 끝났...
2,2,밀정 1930,동양의 파리로 알려진 화려한 1930년대 상해 그러나 화려한 노래와 춤의 이면에는 ...
3,3,엠티맨,첫날 밤 그의 소리가 들리고 둘째 날 밤 그의 모습이 보이고 셋째 날 밤 그가 너를...
4,5,바이크 원정대 인 이탈리아,이탈리아 남부 살레르노를 중심으로바이커들의 로망 이탈리아를 달린다해외여행에 대한 설...


In [6]:
# isnull()
data['본문'].isnull().sum()

6

In [7]:
# fillna()
data['본문'] = data['본문'].fillna('')

In [8]:
data['본문'].isnull().sum()

0

In [9]:
myfile = files.upload()

Saving korean_stopword.csv to korean_stopword.csv


In [10]:
# pd.read_csv로 csv 파일 불러오기
k_stopword = pd.read_csv('/content/korean_stopword.csv')
# list()
stopword = list(k_stopword['불용어'])
stopword[:5]

['이', '있', '하', '것', '들']

In [11]:
stopword = stopword + ['은', '를', '이가']

In [12]:
# del df
# del df['a']
del data['Unnamed: 0']
data.head()

,제목,본문
0,까불지마,벽돌과 개떡 그들은 한 때 잘나가던 동방파의 서열 23위를 다투는 사이 삼복은 그들...
1,엘리트,어두운 학원 교실 조기교육을 받고 있는 아이들이 시험 문제를 풀고 있다 시험이 끝났...
2,밀정 1930,동양의 파리로 알려진 화려한 1930년대 상해 그러나 화려한 노래와 춤의 이면에는 ...
3,엠티맨,첫날 밤 그의 소리가 들리고 둘째 날 밤 그의 모습이 보이고 셋째 날 밤 그가 너를...
4,바이크 원정대 인 이탈리아,이탈리아 남부 살레르노를 중심으로바이커들의 로망 이탈리아를 달린다해외여행에 대한 설...


In [14]:
morphs_lst = []

for i in data.index:
    morphs = ""
    # morphs()
    # data.loc[i]
    morph = okt.morphs(data.loc[i]['본문'])

    for txt in morph:
        if txt not in stopword:
            morphs = morphs + " " + txt

    morphs_lst.append(morphs)

data['morphs'] = morphs_lst
data.head()

,제목,본문,morphs
0,까불지마,벽돌과 개떡 그들은 한 때 잘나가던 동방파의 서열 23위를 다투는 사이 삼복은 그들...,벽돌 과 개떡 잘나가던 동방 파의 서열 23 위 다투는 사이 삼복 의 직속 후배 ...
1,엘리트,어두운 학원 교실 조기교육을 받고 있는 아이들이 시험 문제를 풀고 있다 시험이 끝났...,어 두운 학원 교실 조기교육 을 받고 있는 아이 시험 풀 있다 시험 끝났다는 선생...
2,밀정 1930,동양의 파리로 알려진 화려한 1930년대 상해 그러나 화려한 노래와 춤의 이면에는 ...,동양 의 파리 알려진 화려한 1930년 대 상해 화려한 노래 춤 의 이면 에는 일...
3,엠티맨,첫날 밤 그의 소리가 들리고 둘째 날 밤 그의 모습이 보이고 셋째 날 밤 그가 너를...,첫날 밤 의 들리고 둘째 날 밤 의 모습 셋째 날 밤 너 찾아가리라 10 대 아이...
4,바이크 원정대 인 이탈리아,이탈리아 남부 살레르노를 중심으로바이커들의 로망 이탈리아를 달린다해외여행에 대한 설...,이탈리아 남부 살레르노 중심 으로 바이 커 들의 로망 이탈리아 달린다 해외여행 에...


## TF-IDF(단어 빈도-역 문서 빈도)

[참고](https://m.blog.naver.com/mk_crew/222070694314)

TF-IDF는 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

1. tf(d,t): 특정 문서 d에서의 특정 단어 t의 등장 횟수
2. df(t): 특정 단어 t가 등장한 문서의 수
3. idf(d,t): df(t)에 반비례하는 수

[![](https://mblogthumb-phinf.pstatic.net/MjAyMDA4MjVfMjY3/MDAxNTk4MzQ4MDQzODkz.4YyA2dmV7dmvfGrcJDFP2BSleQ1JG3fBdo8mZkJ1m0Mg.kAADbJSgs1AdNRaZ1fvw9wa5hizTddHn0JP2K-P7lr4g.PNG.mk_crew/image.png?type=w800)](https://m.blog.naver.com/mk_crew/222070694314)|[![](https://mblogthumb-phinf.pstatic.net/MjAyMDA4MjVfMTkx/MDAxNTk4MzQ4MTA1MTc3.LtBgkbBOSw2nuAUH71hZRH2Huvzo9O3N23S-kSgjzUYg.RJ4vl1lJ84Y-KLwtcFPx5_zeZJmTKoV4igKopEQv5Bcg.PNG.mk_crew/image.png?type=w800)](https://m.blog.naver.com/mk_crew/222070694314)
---|---
[![](https://mblogthumb-phinf.pstatic.net/MjAyMDA4MjVfMjIy/MDAxNTk4MzQ4MDgzNDQy.j-ky-EvBulyVxr1eEUeC-C6phqJJOQqjdPBM6Op4b8og.Hob65K7QSRaxHFw0Re1aKwi5xJxDO9TPlOYhfSJaVh4g.PNG.mk_crew/image.png?type=w800)](https://m.blog.naver.com/mk_crew/222070694314)|[![](https://mblogthumb-phinf.pstatic.net/MjAyMDA4MjVfMjcx/MDAxNTk4MzQ4MTE0Nzg5.99wkNQjpnQ3Dbl1XiOzOXy5xEKOHGgFdKX6_C-Pz0akg.Xd5UxL7Kbw4RrGHOMafHiCfHIpGYscDCM9PIXkYEddgg.PNG.mk_crew/image.png?type=w800)](https://m.blog.naver.com/mk_crew/222070694314)

문서1: 먹고 싶은 사과  
문서2: 먹고 싶은 바나나  
문서3: 길고 노란 바나나 바나나  
문서4: 저는 과일이 좋아요  

### TF 계산

In [16]:
import pandas as pd
from math import log

docs = ['먹고 싶은 사과',
        '먹고 싶은 바나나',
        '길고 노란 바나나 바나나',
        '저는 과일이 좋아요']

vocab = list(set(w for doc in docs for w in doc.split()))

In [17]:
vocab

['먹고', '길고', '좋아요', '노란', '싶은', '바나나', '저는', '사과', '과일이']

In [20]:
N = len(docs)

def tf(t, d):
    return d.count(t)

In [21]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))

In [22]:
tf_ = pd.DataFrame(result, columns=vocab)
tf_

,먹고,길고,좋아요,노란,싶은,바나나,저는,사과,과일이
0,1,0,0,0,1,0,0,1,0
1,1,0,0,0,1,1,0,0,0
2,0,1,0,1,0,2,0,0,0
3,0,0,1,0,0,0,1,0,1


### IDF 계산

In [23]:
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N / (df + 1))

In [24]:
result = []

for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
먹고,0.287682
길고,0.693147
좋아요,0.693147
노란,0.693147
싶은,0.287682
바나나,0.287682
저는,0.693147
사과,0.693147
과일이,0.693147


### TF-IDF 계산

In [25]:
def tfidf(t, d):
    return tf(t, d) * idf(t)

In [26]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,먹고,길고,좋아요,노란,싶은,바나나,저는,사과,과일이
0,0.287682,0.000000,0.000000,0.000000,0.287682,0.000000,0.000000,0.693147,0.000000
1,0.287682,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.000000,0.000000
2,0.000000,0.693147,0.000000,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147


## 사이킷런

In [28]:
tfidf = TfidfVectorizer()
# morphs 열에 대해서 tf-idf를 수행
tfidf_matrix = tfidf.fit_transform(data['morphs'])
print(tfidf_matrix.shape)

(16201, 65770)


### 코사인 유사도

- 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미
- 두 벡터의 방향이 완전히 동일한 경우는 1
- 90도의 각을 이루면 0
- 180도로 반대의 방향을 가지면 -1

[![](https://blog.kakaocdn.net/dn/GfONi/btrrcgoMuvR/SvMvIFryQqQnkSbiaa7UK1/img.png)](https://aliencoder.tistory.com/25)

[![](https://images.deepai.org/glossary-terms/cosine-similarity-1007790.jpg)](https://velog.io/@ljs7463/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%ACNLP-%EC%9C%A0%EC%82%AC%EB%8F%84)

왼쪽은 1, 가운데는 0, 오른쪽은 -1  
왼쪽은 벡터의 방향이 동일하기에 유사도가 1이고  높다고 볼 수 있다. 가운데는 유사도가 0이기에 관련이 없고, 오른쪽은 -1이기에 반대라고 볼 수 있다.


In [29]:
# 코사인 유사도를 사용하면 바로 문서의 ()를 구한다
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.00494185, 0.0121967 , ..., 0.00808412, 0.00538158,
        0.01404287],
       [0.00494185, 1.        , 0.0055478 , ..., 0.0069272 , 0.02119022,
        0.00563172],
       [0.0121967 , 0.0055478 , 1.        , ..., 0.00923746, 0.01098523,
        0.00603291],
       ...,
       [0.00808412, 0.0069272 , 0.00923746, ..., 1.        , 0.00703183,
        0.00123542],
       [0.00538158, 0.02119022, 0.01098523, ..., 0.00703183, 1.        ,
        0.00734759],
       [0.01404287, 0.00563172, 0.00603291, ..., 0.00123542, 0.00734759,
        1.        ]])

In [30]:
cosine_sim.shape

(16201, 16201)

In [32]:
indices = pd.Series(data.index, index=data['제목']).drop_duplicates()
print(indices.head())

제목
까불지마              0
엘리트               1
밀정 1930           2
엠티맨               3
바이크 원정대 인 이탈리아    4
dtype: int64


In [33]:
idx = indices['원더 우먼 1984']
print(idx)

27


In [34]:
def get_recommendataions(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]

    movie_indices = []

    for i in sim_scores:
        movie_indices.append(i[0])
    return data['제목'].iloc[movie_indices]

In [35]:
get_recommendataions('조제')

114                   조제
9762                 이퀄스
12976    조제 호랑이 그리고 물고기들
14189         라이 위드 미감독판
16037                교차로
Name: 제목, dtype: object

In [36]:
get_recommendataions('타짜')

11044       스스로 조절하는 삶
3280               직녀성
12206           하이 크라임
6939             사라진 밤
6865     체인징사이드 부부탐구생활
Name: 제목, dtype: object

In [37]:
get_recommendataions('범죄도시')

12574                다이노 타임
5202     극장판 공룡메카드 타이니소어의 섬
1159          백악기 꼬마공룡을 찾아서
10389              다이노X 탐험대
13934      쾌걸 조로리의 공룡알을 지켜라
Name: 제목, dtype: object

In [38]:
get_recommendataions('밀정 1930')

9898         분노의 주먹
5036        600 결사대
4020           상해전기
1423            천여지
6017    야인시대 전쟁의 서막
Name: 제목, dtype: object